In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/FPL Project/my_data/final_dataset.csv')

In [4]:
data['season'].astype(str)

0        1617
1        1617
2        1617
3        1617
4        1617
         ... 
84552    1920
84553    1920
84554    1920
84555    1920
84556    1920
Name: season, Length: 84557, dtype: object

In [5]:
data.dtypes

player                object
gw                     int64
minutes                int64
team                  object
position               int64
position_name         object
opponent_team         object
was_home                bool
total_points           int64
assists                int64
bonus                  int64
bps                    int64
clean_sheets           int64
creativity           float64
goals_conceded         int64
goals_scored           int64
ict_index            float64
influence            float64
own_goals              int64
penalties_missed       int64
penalties_saved        int64
red_cards              int64
saves                  int64
selected               int64
team_a_score         float64
team_h_score         float64
threat               float64
transfers_balance      int64
transfers_in           int64
transfers_out          int64
yellow_cards           int64
kickoff_time          object
season                 int64
xG                   float64
xGA           

In [6]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,gw,minutes,position,was_home,total_points,assists,bonus,bps,clean_sheets,creativity,goals_conceded,goals_scored,ict_index,influence,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,season,xG,xGA,npxG,npxGA
gw,1.000000,-0.049400,0.005535,0.000536,-0.029844,-0.012715,-0.013490,-0.035758,-0.009725,-0.029501,-0.036177,-0.013124,-0.034916,-0.036512,-0.004217,-0.001718,-0.002007,-0.009874,-0.009515,-0.007606,-0.040492,0.015422,-0.020798,0.011794,-0.049428,-0.072060,-0.021685,-0.136705,-0.027801,-0.009279,-0.029433,-0.011589
minutes,-0.049400,1.000000,-0.031178,0.000304,0.634158,0.214454,0.281473,0.760430,0.447195,0.525011,0.624645,0.221648,0.682026,0.704001,0.047871,0.036280,0.038212,0.024514,0.211277,0.339660,0.002494,-0.002756,0.450143,0.110877,0.265619,0.152775,0.284550,0.024123,0.019024,-0.015002,0.020786,-0.012999
position,0.005535,-0.031178,1.000000,0.000106,0.037361,0.080945,0.034716,-0.053710,-0.004301,0.161656,-0.019873,0.146093,0.151393,-0.035072,-0.019727,0.030480,-0.051121,0.003262,-0.284141,0.002622,-0.000533,0.003209,0.247813,0.005176,0.081217,0.085365,0.018249,-0.028047,-0.026544,0.025681,-0.028051,0.025270
was_home,0.000536,0.000304,0.000106,1.000000,0.045398,0.023079,0.027715,0.030664,0.063269,0.041674,-0.066505,0.024187,0.038817,0.011730,-0.008025,0.005264,-0.005669,-0.004315,-0.013066,0.002197,0.006964,-0.007460,0.043135,0.016824,0.013316,-0.007498,-0.011038,-0.000092,0.001570,-0.004473,0.001403,-0.004397
total_points,-0.029844,0.634158,0.037361,0.045398,1.000000,0.454561,0.750488,0.906389,0.582345,0.471532,0.163786,0.663364,0.759519,0.831346,-0.024200,0.016839,0.100930,-0.043346,0.166201,0.316849,-0.014408,-0.002717,0.546326,0.105507,0.253537,0.146263,0.068703,0.013355,0.097873,-0.092232,0.100368,-0.090906
assists,-0.012715,0.214454,0.080945,0.023079,0.454561,1.000000,0.305168,0.397770,0.127668,0.403109,0.096744,0.114514,0.409483,0.374497,0.004204,0.014940,-0.005241,0.006753,-0.026916,0.142287,0.068852,0.075051,0.245194,0.049746,0.130012,0.080867,0.050695,0.003906,0.069895,-0.055318,0.070447,-0.055404
bonus,-0.013490,0.281473,0.034716,0.027715,0.750488,0.305168,1.000000,0.642968,0.279379,0.301935,0.037163,0.615270,0.532717,0.621008,-0.001502,0.013725,0.108000,-0.007987,0.102021,0.181826,-0.001793,-0.004423,0.370111,0.062859,0.152548,0.088838,0.024966,0.006169,0.059783,-0.055443,0.060841,-0.055057
bps,-0.035758,0.760430,-0.053710,0.030664,0.906389,0.397770,0.642968,1.000000,0.505269,0.525638,0.353394,0.567929,0.778996,0.905825,0.007899,0.006998,0.081241,-0.026188,0.226784,0.316102,0.019452,0.031791,0.474648,0.102413,0.238177,0.132963,0.126828,0.004900,0.094679,-0.085907,0.096911,-0.085398
clean_sheets,-0.009725,0.447195,-0.004301,0.063269,0.582345,0.127668,0.279379,0.505269,1.000000,0.281883,-0.181526,0.128180,0.349951,0.336813,-0.013253,0.020695,0.007530,0.000609,0.068201,0.196098,-0.166616,-0.136352,0.243915,0.080988,0.160383,0.073345,0.103825,-0.000432,0.085378,-0.100142,0.088523,-0.098285
creativity,-0.029501,0.525011,0.161656,0.041674,0.471532,0.403109,0.301935,0.525638,0.281883,1.000000,0.273225,0.224759,0.775104,0.501018,0.005458,0.038180,-0.011772,0.013506,-0.064890,0.266727,0.008241,0.013887,0.471547,0.099190,0.249084,0.149700,0.151192,0.003912,0.103587,-0.089940,0.105211,-0.090616


In [7]:
cat_vars = ['player', 'gw', 'position', 'team', 'opponent_team', 'was_home','position_name']
cont_vars = ['minutes','xG','xGA','npxG','npxGA', 'bps', 'creativity', 'ict_index', 'influence', 'selected','threat','transfers_balance','total_points'] 
my_cols = cat_vars + cont_vars

In [8]:
my_cols

['player',
 'gw',
 'position',
 'team',
 'opponent_team',
 'was_home',
 'position_name',
 'minutes',
 'xG',
 'xGA',
 'npxG',
 'npxGA',
 'bps',
 'creativity',
 'ict_index',
 'influence',
 'selected',
 'threat',
 'transfers_balance',
 'total_points']

In [9]:
data = data[my_cols]

In [10]:
from sklearn.model_selection import train_test_split

y = data['total_points']

X = data.drop(data[[ 'total_points']], axis =1)# dropping columns that are not relevant and the prediction column

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [11]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['player', 'team', 'opponent_team', 'position_name']


In [12]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [13]:
OH_X_train

,gw,position,was_home,minutes,xG,xGA,npxG,npxGA,bps,creativity,ict_index,influence,selected,threat,transfers_balance,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,1291,1292,1293,1294,1295,1296,1297,1298,1299,1300,1301,1302,1303,1304,1305,1306,1307,1308,1309,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319,1320,1321,1322,1323,1324,1325,1326,1327,1328,1329,1330
25882,5,3,True,0,35.184401,48.993914,34.423230,48.232744,0,0.0,0.0,0.0,13817,0.0,-888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
5113,9,1,False,0,34.223680,65.663528,30.417833,62.618848,0,0.0,0.0,0.0,20667,0.0,-981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
36780,24,2,False,0,45.847429,53.981191,41.129107,49.414176,0,0.0,0.0,0.0,535,0.0,-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
55392,17,3,False,33,53.866134,49.309790,49.922344,45.396307,2,14.8,2.5,6.0,232763,4.0,-25417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12516,22,2,False,90,39.499543,65.129661,37.216034,58.923443,15,0.3,3.7,26.6,2084,10.0,-111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59709,25,3,False,13,53.058692,42.691294,50.014016,39.646622,3,0.6,0.1,0.4,25393,0.0,-1031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
54746,16,3,False,0,29.002263,67.194016,28.241098,61.104739,0,0.0,0.0,0.0,117869,0.0,-17119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
44310,36,3,True,0,59.043412,43.540152,56.631119,42.778982,0,0.0,0.0,0.0,49589,0.0,-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
32362,16,2,False,0,35.184401,48.993914,34.423230,48.232744,0,0.0,0.0,0.0,429,0.0,284,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [14]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(OH_X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(OH_X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = OH_X_train.columns
imputed_X_valid.columns = OH_X_valid.columns

In [15]:
from xgboost import XGBRegressor

my_model_1 = XGBRegressor()
my_model_1.fit(imputed_X_train, y_train)

[11:59:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [16]:
# Get predictions
predictions_1 = my_model_1.predict(imputed_X_valid) 

In [17]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

# Calculate MAE
mae_1 = mean_absolute_error(y_valid, predictions_1)
print("Mean Absolute Error:" , mae_1)

Mean Absolute Error: 0.3331269008011444


**Improving the model**

In [18]:
# Define the model
my_model_2 = XGBRegressor(gamma=0.047479, learning_rate=0.001, max_depth=5, n_estimators=100, subsample=0.6862963240045978) # Your code here

# Fit the model
my_model_2.fit(imputed_X_train, y_train,
              early_stopping_rounds = 5,
              eval_set =[(imputed_X_valid,y_valid)],
              verbose=False) 

[12:02:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.047479,
             importance_type='gain', learning_rate=0.001, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.6862963240045978, verbosity=1)

In [19]:
# Get predictions
predictions_2 = my_model_2.predict(imputed_X_valid)

In [24]:
# Calculate MAE
mae_2 = mean_absolute_error(y_valid, predictions_2)
print("Mean Absolute Error:" , mae_2)

Mean Absolute Error: 1.2954929631587573
